<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/navernews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 37.0 MB/s eta 0:00:00


**News Crawler**

In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, page=1, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    pages = list(range(1,page+1))
    li = []
    for page in pages:
      url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")

      div = bs_obj.find_all('div', {'class':'hotNewsList'})
      li+=[html.unescape(x['href']) for x in div[0].find_all('a')]
    return li

  def get_main(self, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/mainnews.nhn'
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return [x.find_all('a')[0]['href'] for x in bs_obj.find_all('dd', {'class':'articleSubject'})]

  # 시황 전망: 401
  # 기업 종목분석: 402
  # 해외 증시: 403
  # 채권 선물: 404
  # 공시 메모: 406
  # 환율: 429
  def get_focus(self, page=1, code=402, date='1', ):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    pages = range(0,page)
    li = []
    for page in pages:
      url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3='\
      + str(code) +  '&date=' + str(date) + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")
      temp = [html.unescape(x.find_all('a')[0]['href']) for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
      # 업데이트 가능
      li+=[x.replace('§', '&sect') for x in temp]
    return li

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    text = text.replace(subheading, '')
    return title, subheading, text

  def get_code(self, code):
    url = 'https://finance.naver.com/item/news_news.naver?code=' + code + '&page=1&sm=title_entity_id.basic&clusterId='
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    tds = bs_obj.find_all('td', {'class': 'title'})
    texts = [td.text.strip() for td in tds]
    cleaned =  [sentence.replace('[', ' ').replace(']', ' ').replace('…', ' ').replace('·', ' ').replace('"', ' ').replace('“', ' ').replace(',', ' ').replace('...', ' ') for sentence in texts]
    return cleaned


In [25]:
from multiprocessing import Pool

news = News()
news_list = news.get_ranking(5)
with Pool() as p:
    li = p.map(news.get_article, news_list)
li = np.array(li)

dic = {'Title':li[:,0], 'Subheading':li[:,1], 'Text':li[:,2]}
df = pd.DataFrame.from_dict(dic)
df

,Title,Subheading,Text
0,"""지금 이 주식 사세요""…유명 애널의 추천, 이유 있었다","미리 사놓고 '매수' 리포트5.2억 챙긴 증권사 애널리스트금감원 특사경, 기소의견으...",서울 여의도에 있는 금융감독원 전경. /사진=한경DB금융감독원 자본시장특별사법경찰(...
1,"“창고가 텅텅 비었어요”…활짝 웃은 이 업종, 주가도 뛰었다",,국내 의류 OEM주 강세 [사진 = 연합뉴스]영원무역과 한세실업 등 주문자상표부착생...
2,숨고르는 코스피…저평가株 담아볼까,긴축 우려에 코스피 단기 조정장세 지속상승 재료 부재에 저PER 종목 관심 기대은행...,[이데일리 김응태 기자] 코스피가 글로벌 긴축 정책과 경기 침체 우려에 숨고르기 구...
3,美 억만장자 투자자의 깜짝 놀랄 만한 증시 전망,,미국의 억만장자 투자자이자 배런 캐피털의 최고경영자 론 배런이 인플레이션 상승에 따...
4,"SK이노베이션 ""자사주 소각 결정된 바 없어""[주목 e공시]",,"SK이노베이션은 자사주 소각 추진 보도에 대해 ""주주가치 제고를 위해 자사주 소각을..."
...,...,...,...
118,"씨에스베어링, 전방 호조 속 경쟁자 품질 이슈에 수혜 전망…목표가↑-신한","투자의견 ‘매수’, 목표가 1만9000원 ‘상향’",[이데일리 이정현 기자] 신한투자증권은 씨에스베어링(297090)에 대해 “경쟁사 ...
119,"위안화 약세 vs 반기말 네고…환율, 1300원 후반 등락 전망[외환브리핑]",역외 1304.0원…하락 출발 전망위안화 작년 11월 이후 최대 약세달러인덱스 10...,[이데일리 하상렬 기자] 원·달러 환율이 소폭 하락 후 중국 위안화 약세에 연동된 ...
120,"[특징주]TS트릴리온, 최대주주 경영권 매각 소식에 강세",,[이데일리 양지윤 기자] TS트릴리온이 최대주주의 경영권 매각 소식에 강세다. 27...
121,"미래에셋운용, 'TIGER Fn반도체TOP10 ETF' 순자산 4천억원 돌파",,[이데일리 이은정 기자] 미래에셋자산운용은 ‘TIGER Fn반도체TOP10 ETF(...


In [ ]:
df

In [28]:
# korean font
!wget 'https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKkr-hinted.zip'
!unzip "NotoSansCJKkr-hinted.zip"
!mv NotoSansCJKkr-Medium.otf /usr/share/fonts/truetype/

--2023-06-27 05:33:00--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKkr-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 74.125.137.128, 2607:f8b0:4023:c03::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|74.125.137.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121163981 (116M) [application/zip]
Saving to: ‘NotoSansCJKkr-hinted.zip.1’

NotoSansCJKkr-hinte 100%[===================>] 115.55M   168MB/s    in 0.7s    

2023-06-27 05:33:00 (168 MB/s) - ‘NotoSansCJKkr-hinted.zip.1’ saved [121163981/121163981]

Archive:  NotoSansCJKkr-hinted.zip
replace LICENSE_OFL.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: LICENSE_OFL.txt         
replace NotoSansCJKkr-Black.otf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NotoSansCJKkr-Black.otf  
replace NotoSansCJKkr-Bold.otf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: NotoS

In [34]:
# https://wikidocs.net/22530
from konlpy.tag import Hannanum
from wordcloud import WordCloud
import matplotlib.pyplot as plt

class Analyser:
  def __init__(self, df):
    self.df = df
    self.words = ''
    self.stop_words = ['있다', '기자', '다만', '등의', '것으로', '기대감', '기대감에', '특징주', '실적', '기대', '최대', '신영', '순매수', '상위종목', '마감', \
                  '코스닥', '규모', '기관', '외국인', '신영증권', '이데일리', '연합뉴스', '서울경제', '하이투자증권', 'NH투자증권', '뉴스1', '영상', '서울', \
                  '뉴시스', '고', '며', '포인트', '라', '사진', '현지시간', 'S', 'SP', '헤럴드경제', '사진제공', '것', '이라', '로부터', '때문', '제공', 'P', '애널리스트']
    self.wordcloud = ''

  def get_tokenized(self, text):
    hannanum = Hannanum()
    return hannanum.morphs(text)

  def get_cleaned(self, text):
    text = ([x for x in text if len(x) > 1])
    d = {i:text.count(i) for i in set(text) if text.count(i) > 1}
    words = ' '.join(d.keys())
    return words

  def get_wordcloud(self, column):
    t = analyser.df[column]
    words = self.get_tokenized(' '.join(t))
    words = self.get_cleaned(words)
    words = words.split()
    words = ' '.join(words)
    self.words = words
    wordcloud = WordCloud(stopwords = self.stop_words, \
                          font_path='/usr/share/fonts/truetype/NotoSansCJKkr-Medium.otf', \
                          background_color='white', \
                          width=1200, height=800).generate(words)
    self.wordcloud = wordcloud
    plt.figure(figsize=(18,10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")

In [59]:
news = News()
s = news.get_code('007660')

In [60]:
s

[' 주식초고수는 지금  AI 반도체 수혜株 이수페타시스 순매수 1위',
 ' AI 반도체 수혜  이수페타시스 신고가 행진',
 ' 주가동향  이수페타시스 20% 상승세  구글 AI가속기 수주 물량 ',
 " 주식 초고수는 지금 '엔비디아 효과' 이수페타시스 순매수 1위",
 " MK 시그널  'AI수혜주' 이수페타시스 주목",
 'SK證  이수페타시스  시장 확대로 실적 개선 ',
 ' 이번주 추천주  엔비디아發 수혜 이수페타시스 주목',
 ' 데이터로 보는 증시 이수페타시스  SK하이닉스  기관 외국인 코스피  ',
 ' 우리도 AI 수혜주” 이수페타시스 급등',
 '이수페타시스  고다층 PCB 4공장 준공식 개최',
 '이수페타시스  4공장 준공식 연간 2천억 규모 PCB 추가 생산능력  ',
 ' 데이터센터 수요에 선제 대응” 이수페타시스  4공장 준공']

In [55]:
cleaned_sentences =

SyntaxError: ignored